### Top Coins

In [1]:
from curl_cffi import requests
from typing import List
from pydantic import BaseModel
import pandas as pd

params = {
    "start": 1,
    "limit": 100,
    "sortBy": "rank",
    "sortType": "desc",
    "convert": "USD,BTC,ETH",
    "cryptoType": "all",
    "tagType": "all",
    "audited": "false",
    "aux": "cmc_rank,date_added",
    "marketCapRange": "100000000~",
    "volume24hRange": "1000000~",
    "ageRange": "4204800~",
}
r = requests.get(
    "https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing",
    params=params,
    impersonate="chrome",
)
data = r.json()


class CryptoCurrency(BaseModel):
    id: int
    name: str
    symbol: str
    dateAdded: str


class CryptoData(BaseModel):
    cryptoCurrencyList: List[CryptoCurrency]


class ApiResponse(BaseModel):
    data: CryptoData


parsed = ApiResponse(**data)
df = pd.DataFrame(
    [
        {
            "name": coin.name.lower(),
            "symbol": coin.symbol.upper(),
        }
        for coin in parsed.data.cryptoCurrencyList
    ]
)

# df = df[df["symbol"] != "usdt"]
df = df.set_index("name")

In [2]:
from datetime import date
from concurrent.futures import ThreadPoolExecutor, as_completed
from binance.download_kline import download_monthly_klines

# 🔹 Build symbols list (exclude USDT itself)
symbols = [f"{sym}USDT" for sym in df["symbol"] if sym.upper() != "USDT"]

# 🔹 Fixed start and end dates
start_date = date(2017, 1, 1)
end_date = date.today()
print("Downloading from", start_date, "to", end_date)

# 🔹 Configuration
MAX_WORKERS = 4


def download_symbol(symbol):
    """Download klines for a single symbol."""
    try:
        print(f"\n🚀 Starting download for {symbol} ...")
        download_monthly_klines(
            trading_type="spot",
            symbols=[symbol],
            num_symbols=1,
            intervals=["1h"],
            years=range(start_date.year, end_date.year + 1),
            months=range(1, 13),
            start_date=start_date.strftime("%Y-%m-%d"),
            end_date=end_date.strftime("%Y-%m-%d"),
            folder="./data",
            checksum=0,
        )
        return symbol, "success"
    except Exception as e:
        return symbol, str(e)


with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_symbol = {
        executor.submit(download_symbol, symbol): symbol for symbol in symbols
    }

    for future in as_completed(future_to_symbol):
        symbol, result = future.result()
        if result == "success":
            print(f"✅ Completed {symbol}")
        else:
            print(f"❌ Failed {symbol}: {result}")

print("\n✅ All downloads completed (or failed).")


🚀 Starting download for BTCUSDT ...
Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

🚀 Starting download for ETHUSDT ...
Found 1 symbols
[1/1] - start download monthly ETHUSDT klines 

🚀 Starting download for BNBUSDT ...
Found 1 symbols
[1/1] - start download monthly BNBUSDT klines 

🚀 Starting download for XRPUSDT ...
Found 1 symbols
[1/1] - start download monthly XRPUSDT klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BNBUSDT/1h/BNBUSDT-1h-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1h/BTCUSDT-1h-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1h/XRPUSDT-1h-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/ETHUSDT/1h/ETHUSDT-1h-2017-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1h/XRPUSDT-1h-2017-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSDT/1h